# Finetuning AlexNet

In [2]:
import os
import sys
import numpy
import math
import caffe

from pprint import pprint
from pylab import *
%matplotlib inline

import pandas as pd

# helper to show images in square
# take an array of shape (n, height, width) or (n, height, width, channels)
# and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)
def vis_square(data, padsize=1, padval=0, canvas=plt):
    for im in data:
        pass
        #im -= im.min()
        #im /= im.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    if data.ndim == 4:
        canvas.imshow(data)
    else:
        canvas.imshow(data, cmap='gray')
        
# Grab descriptions of the labels
classes = pd.read_csv('temp/class-list.csv', index_col='label', header=0)
print(classes)

caffe.set_device(0)
caffe.set_mode_gpu()

TRAIN_BATCH = 256
TEST_BATCH = 256

                        description
label                              
0                           No Data
1                        Open Water
2                Perennial Ice/Snow
3             Developed; Open Space
4          Developed; Low Intensity
5       Developed; Medium Intensity
6          Developed High Intensity
7      Barren Land (Rock/Sand/Clay)
8                  Deciduous Forest
9                  Evergreen Forest
10                     Mixed Forest
11                      Dwarf Scrub
12                      Shrub/Scrub
13             Grassland/Herbaceous
14                 Sedge/Herbaceous
15                          Lichens
16                             Moss
17                      Pasture/Hay
18                 Cultivated Crops
19                   Woody Wetlands
20     Emergent Herbaceous Wetlands


In [3]:
def define_solver(base_lr, lr_step, gamma, weight_decay, momentum, snapshot_prefix, snapshot_iters, **kwargs):
    return """# Generated from python.
    # Train Set
    #
    # Training net
    train_net: "temp/auto-train.prototxt"
    # Test net
    test_net: "temp/auto-test.prototxt"
    # These numbers don't matter, because we're running the test net ourselves.
    test_iter: 1
    test_interval: 100000
    # The base learning rate, momentum and the weight decay of the network.
    base_lr: %(base_lr)s
    momentum: %(momentum)i
    weight_decay: %(weight_decay)s
    # The learning rate policy
    lr_policy: "step"
    gamma: %(gamma)f
    stepsize: %(lr_step)i
    display: 500
    # The maximum number of iterations
    max_iter: 10000
    # snapshot intermediate results
    snapshot: %(snapshot_iters)i
    snapshot_prefix: "%(snapshot_prefix)s"
    # solver mode: CPU or GPU
    solver_mode: GPU""" % locals()

from caffe import layers as L
from caffe import params as P

def define_net(lmdb, lmdb_labels, batch_size, mean_file, **kwargs):
    with open('finetune/train_val.prototxt') as f:
        return f.read() % locals()

## Set up the network
def create_solver(train_data, train_labels, **kwargs):
    with open('temp/autosolver.prototxt', 'w') as f:
        f.write(define_solver(**kwargs))

    with open('temp/auto-train.prototxt', 'w') as f:
        f.write('name: "SeedNet"\n')
        f.write(str(define_net(train_data, train_labels, TRAIN_BATCH,
                               train_data + '_mean.binaryproto',
                              **kwargs)))

    with open('temp/auto-test.prototxt', 'w') as f:
        f.write('name: "SeedNet"\n')
        f.write(str(define_net('temp/learnsat-val-lmdb',
                               'temp/classify/val-labels-lmdb',
                               TEST_BATCH,
                               train_data + '_mean.binaryproto',
                              **kwargs)))

        
    solver = caffe.SGDSolver('temp/autosolver.prototxt')
    solver.net.copy_from(os.path.join(os.environ['CAFFE_ROOT'], 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'))
    solver.test_nets[0].copy_from(os.path.join(os.environ['CAFFE_ROOT'], 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'))

    
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( train_data + '_mean.binaryproto' , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    mean_image = arr[0]

    # input preprocessing: 'data' is the name of the input blob == net.inputs[0]
    transformer = caffe.io.Transformer({'data': solver.net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', mean_image)
    
    return solver, transformer


import sklearn.metrics as metrics
def run_test(net, iterations):
    y_true = []
    y_pred = []
    for test_it in range(iterations):
        net.forward()
        y_pred.extend(net.blobs['fc8_sat'].data.argmax(1))
        y_true.extend(net.blobs['labels'].data.reshape(net.blobs['fc8_sat'].data.shape).argmax(1))
    
    accuracy_score = metrics.accuracy_score(y_true, y_pred)
    precision_score = metrics.precision_score(y_true, y_pred, average='macro')
    recall_score = metrics.recall_score(y_true, y_pred, average='macro')
    
    return accuracy_score, precision_score, recall_score


import json

def run(prefix, **params):
    print('Running with params: ' + str(params))
    TRAIN_DATA = 'temp/learnsat-subset-' + str(params['train_size']) + '-train-lmdb'
    TRAIN_LABELS = 'temp/classify/train-' + str(params['train_size']) + '-labels-lmdb'
    TEST_SIZE = 4000
    TEST_EPOCH = TEST_SIZE / TEST_BATCH

    train_epoch = params['train_size'] / TRAIN_BATCH
    
    # convert lr_step from iterations to epochs
    params['lr_step'] = params['lr_step'] * train_epoch
    
    niter = train_epoch * params['n_epochs'] + 1

    hp = '.'.join(['-'.join((p, str(params[p]))) for p in params])
    params['snapshot_prefix'] = prefix + '.snap.' + hp + '.'
    params['snapshot_iters'] = niter - 1

    solver, transformer = create_solver(TRAIN_DATA,TRAIN_LABELS,**params)

    test_interval = train_epoch
    ntest = 1 + int(np.ceil(float(niter-1) / test_interval))
    train_loss = zeros(niter)
    train_acc = zeros(ntest)
    train_precision = zeros(ntest)
    train_recall = zeros(ntest)
    test_acc = zeros(ntest)
    test_precision = zeros(ntest)
    test_recall = zeros(ntest)
    output = zeros((niter, 16, 21))

    # the main solver loop
    for it in range(niter):
        solver.step(1)  # SGD by Caffe

        # store the train loss
        train_loss[it] = solver.net.blobs['loss'].data

        # run a full test every so often
        # (Caffe can also do this for us and write to a log, but we show here
        #  how to do it directly in Python, where more complicated things are easier.)
        if it % test_interval == 0:
            index = it // test_interval
            test_acc[index], test_precision[index], test_recall[index] = run_test(solver.test_nets[0], TEST_EPOCH)
            train_acc[index], train_precision[index], train_recall[index] = run_test(solver.net, train_epoch)
    
    # save results to file
    if prefix:
        last_acc = test_acc[-1]
        filePrefix = prefix + '.val-' + str(last_acc) + '.' + hp
        with open(filePrefix + '.params.json', 'w') as f:
            f.write(json.dumps(params))
        np.save(filePrefix + '.train_loss.npy', train_loss)
        np.save(filePrefix + '.train_acc.npy', train_acc)
        np.save(filePrefix + '.val_acc.npy', test_acc)

    return (
        solver,
        transformer,
        params,
        niter,
        train_epoch,
        test_interval,
        train_loss,
        train_acc,
        train_precision,
        train_recall,
        test_acc,
        test_precision,
        test_recall)


def plot_result(
        params,
        niter,
        train_epoch,
        test_interval,
        train_loss,
        train_acc,
        train_precision,
        train_recall,
        test_acc,
        test_precision,
        test_recall):
    
    fig, (ax1, bx1) = subplots(1, 2, figsize=(12,5))
    fig.suptitle(str(params))
    ax2 = ax1.twinx()
    ax1.plot(arange(niter) / float(train_epoch), train_loss)
    ax2.plot(test_interval * arange(len(test_acc)) / train_epoch, test_acc, 'r',
             test_interval * arange(len(train_acc)) / train_epoch, train_acc, 'g')
    #ax3 = ax2.twinx()
    #ax3.plot(test_interval * arange(len(test_acc)), train_acc, 'g')
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('train loss')
    ax1.set_ylim((0,16))
    ax2.set_ylabel('accuracy')
    ax2.set_ylim((0,1))
    #ax3.set_ylabel('test accuracy')

    fscore = test_precision * test_recall / (test_precision + test_recall)
    bx2 = bx1.twinx()
    bx1.plot(arange(niter), train_loss)
    bx2.plot(test_interval * arange(len(test_acc)), test_precision, 'r',
             test_interval * arange(len(train_acc)), test_recall, 'y',
             test_interval * arange(len(train_acc)), fscore, 'g')
    #ax3 = ax2.twinx()
    #ax3.plot(test_interval * arange(len(test_acc)), train_acc, 'g')
    bx1.set_xlabel('iteration')
    bx1.set_ylabel('train loss')
    bx1.set_ylim((0,None))
    bx2.set_ylabel('precision, recall, fscore')
    bx2.set_ylim((0,1))
    #ax3.set_ylabel('test accuracy')
    return fig

## Hyperparameter Search

In [4]:
rates = [10 ** uniform(-4, -1) for x in range(4)]
lambdas = [10 ** uniform(-3, 0) for x in range(4)]
params = [
    dict(
        n_epochs=60,
        train_size=10240,
        weight_decay=wd,
        base_lr=lr,
        momentum=mu,
        gamma=0.1,
        lr_step=lr_step
    )
    for lr in rates
    for lr_step in [5, 10, 20, 40]
    for wd in lambdas
    for mu in [0.5, 0.9, 0.95, 0.99]
]

results = [run('finetune/search/alex', **hp)[2:] for hp in params[:1]]

Running with params: {'lr_step': 5, 'n_epochs': 1, 'train_size': 10240, 'base_lr': 0.00826074106984656, 'momentum': 0.5, 'weight_decay': 0.04840675221764321, 'gamma': 0.1}


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
plot_result(*result[2:])
pass

[{'base_lr': 0.004157626033358791,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.04069495718604703},
 {'base_lr': 0.004157626033358791,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.025323647758374014},
 {'base_lr': 0.004157626033358791,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.0012418741132024511},
 {'base_lr': 0.004157626033358791,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.001277685015719408},
 {'base_lr': 0.0074787508611772885,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.13255204871387155},
 {'base_lr': 0.0074787508611772885,
  'gamma': 0.1,
  'lr_step': 20,
  'momentum': 0.9,
  'n_epochs': 5,
  'train_size': 4096,
  'weight_decay': 0.007865003196694803

In [ ]:
fig, ax = subplots()
losses = [res[4][-1] for res in results]
accuracy = [res[8][-1] for res in results]
lambs = [res[0]['weight_decay'] for res in results]
ax.plot(lambs, accuracy, 'g')
ax.set_ylim((0, 1))

In [ ]:
labels = classes.index.tolist()
descriptions = classes.description.tolist()
def report(net, iterations):
    y_true = []
    y_pred = []
    for test_it in range(iterations):
        net.forward()
        y_pred.extend(net.blobs['ip2'].data.argmax(1))
        y_true.extend(net.blobs['reshape_labels'].data.argmax(1))
    
    print(metrics.confusion_matrix(y_true, y_pred))
    print(metrics.classification_report(y_true, y_pred, labels, descriptions))

report(solver.net, TRAIN_EPOCH)

## Sanity Checks

After loading up a solver, run these checks to see if it's set up as expected.

In [ ]:
solver = result[0]
transform = result[1]

solver.net.forward()
# To get an idea of the architecture of our net, we can check the dimensions of 
# the intermediate features (blobs) and parameters (these will also be useful to 
# refer to when manipulating data later).

# each output is (batch size, feature dim, spatial dim)
pprint([(k, v.data.shape) for k, v in solver.net.blobs.items()])

# just print the weight sizes (not biases)
pprint([(k, v[0].data.shape) for k, v in solver.net.params.items()])

# now draw the network!
!$CAFFE_ROOT/python/draw_net.py --rankdir TB temp/auto-train.prototxt temp/net.png
from IPython.display import Image
Image(filename='temp/net.png', width=300)

In [ ]:
## Baseline accuracy.
solver.net.forward()  # train net
solver.test_nets[0].forward()  # test net (there can be more than one)
print(solver.net.blobs['fc8_sat'].data[0])
print(solver.net.blobs['labels'].data[0])
print(solver.net.blobs['loss'].data)
# run an initial forward pass for the full dataset, and see what kind of results we get.
# accuracy should be ~ 1/num-classes
print(run_test(solver.net, 10240 / TRAIN_BATCH))
print(run_test(solver.test_nets[0], 4000 / TEST_BATCH))

Are we loading train and test images, and their labels?

In [ ]:
#solver.net.forward()
index = 0
count = 16
images = solver.net.blobs['data'].data[index:index+count]
images = numpy.array([transform.deprocess('data', i) for i in images])
vis_square(images, 1, 0)
labels = solver.net.blobs['labels'].data.reshape(TRAIN_BATCH, 21)[index:index+count]
pd.DataFrame(labels, columns=classes['description'].tolist())

In [ ]:
from IPython.display import display
from random import randint

def softmax(w, t = 1.0):
    e = np.exp(np.array(w) / t)
    dist = e / np.sum(e)
    return dist

def top5(data):
    df = pd.DataFrame(data, columns=classes['description'].tolist())
    return df.T.sort(ascending=False, columns=[0])[:6].T


index = randint(0, solver.test_nets[0].blobs['data'].data.shape[0])
N = 1

print(index)

images = solver.test_nets[0].blobs['data'].data[index:index+N]
images = numpy.array([transform.deprocess('data', img) for img in images])
_, ax = subplots()
vis_square(images, 1, 0, ax)

row = np.array(['Actual ' + str(i) for i in range(index, index+N)])
labels = solver.test_nets[0].blobs['labels'].data.reshape(TEST_BATCH, 21)[index:index + N]
predictions = softmax(solver.test_nets[0].blobs['fc8_sat'].data[index:index + N])

display(top5(labels))
display(top5(predictions))

Both train and test nets seem to be loading data, and to have correct labels. Let's take one step of (minibatch) SGD and see what happens.

Do we have gradients propagating through our filters? Let's see the updates to the first layer, shown here as a $4 \times 5$ grid of $5 \times 5$ filters.

In [ ]:
solver.step(1)
print(solver.net.params['conv1'][0].diff[:, 0].shape)
vis_square(solver.net.params['conv1'][0].diff[:, 0])

## Visualizing Filters

After some training, we can visualize what the filters of our convnet look like, and what kinds of outputs they produce

In [ ]:
index = 0
plt.rcParams['figure.figsize'] = (5,5)
figure()
vis_square(numpy.array([transform.deprocess('data', solver.net.blobs['data'].data[index])]))

fig, (ax1, ax2) = subplots(1, 2)
filters = solver.net.params['conv1'][0].data
vis_square(filters.transpose(0, 2, 3, 1), 1, 0, ax1)
images = solver.net.blobs['conv1'].data[index]
vis_square(images, 1, 0, ax2)

fig, (ax1, ax2) = subplots(1, 2)
solver.net.params['conv2'][0].data.shape
vis_square(solver.net.params['conv2'][0].data[:48].reshape(48**2, 5, 5), 2, 0, ax1)
vis_square(solver.net.blobs['conv2'].data[index], 2, 0, ax2)

#fig, (ax1, ax2) = subplots(1, 2)
#vis_square(solver.net.params['conv3'][0].data[:40].reshape(40**2, 3, 3), 2, 0, ax1)
#vis_square(solver.net.blobs['conv3'].data[index], 1, 0, ax2)
